In [16]:
import re
from typing import cast

import bs4
import ocp_client
import utils
from rich.progress import track

In [17]:
def get_maker_urls(subcat_url: str):
    soup = utils.soup(subcat_url)
    return [utils.get_link(el) for el in soup.select("a.top-auto-item__image")]

def parse_vehicle_permalink(permalink: str):
    r = re.match(r"https://www.onlinecarparts.co.uk/spare-parts/(.+)/brake-pads/(.+).html", "https://www.onlinecarparts.co.uk/spare-parts/bmw/brake-pads/3-e90.html")
    maker, model = r.groups()
    return f"{maker}/{model}"

def get_vehicles(maker_url: str):
    soup = utils.soup(maker_url)
    return [
        (
            parse_vehicle_permalink(utils.get_link(el)),
            cast(str, el.select_one("img")["src"]),
        )
        for el in soup.select("a.top-auto-item__image")
    ]

In [18]:
soup = utils.soup("https://www.onlinecarparts.co.uk/spare-parts.html")
subcats = [
    (
        el.select_one(".catalog-subcats__name").text,
        utils.get_link(el),
    )
    for el in soup.select("a.catalog-subcats__item")
]

maker_urls = get_maker_urls(subcats[0][1])
vehicles = get_vehicles(maker_urls[0])
vehicles[0]

('bmw/3-e90', 'https://scdn.autoteiledirekt.de/groups/170x100/952.png')